In [21]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

## Setting up the basics

In [22]:
mnist = input_data.read_data_sets('./inputs/mnist')
# Resetting default graph, starting from scratch
tf.reset_default_graph()

batch_size = 64
n_noise = 200
learning_rate=0.00015

X_in = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1], name='X')
noise = tf.placeholder(dtype=tf.float32, shape=[None, n_noise])

# The keep_prob variable will be used by our dropout layers, which we introduce for more stable learning outcome
keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
is_training = tf.placeholder(dtype=tf.bool, name='is_training')

# Leaky Relu activation
# https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29#Potential_problems
def lrelu(x):
    return tf.maximum(x, tf.multiply(x, 0.2))

# Binary cross entropy for descriminators
def binary_cross_entropy(x, z):
    eps = 1e-12
    return (-(x * tf.log(z + eps) + (1. - x) * tf.log(1. - z + eps)))

Extracting ./inputs/mnist/train-images-idx3-ubyte.gz
Extracting ./inputs/mnist/train-labels-idx1-ubyte.gz
Extracting ./inputs/mnist/t10k-images-idx3-ubyte.gz
Extracting ./inputs/mnist/t10k-labels-idx1-ubyte.gz


# The descriminator

In [23]:
# It takes either real or fake MNIST image 28 x 28 in grayscale
# we use a sigmoid to make sure our output can be interpreted 
# as the probability the input image is a real MNIST character.
def discriminator(X_in, reuse=None, keep_prob=keep_prob):
    activation=lrelu
    with tf.variable_scope('disc', reuse=reuse):
        x = tf.reshape(X_in, shape=[-1, 28, 28, 1])
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, units=128, activation=activation)
        x = tf.layers.dense(x, units=1, activation=tf.nn.sigmoid)
        return x

# The generator

In [24]:
# z => noise
def generator(z, keep_prob=keep_prob, is_training=is_training):
    activation = lrelu
    momentum = 0.99
    with tf.variable_scope('gen', reuse=None):
        # Simply reassigning noise to x to begin with
        x = z
        d1 = 4
        d2 = 1
        print('checking units ', (d1 * d1 * d2))
        x = tf.layers.dense(x, units=d1 * d1 * d2, activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        # https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.reshape(x, shape=[-1, d1, d1, d2])
        x = tf.image.resize_images(x, size=[7, 7])
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        return x

# Loss functions and optimizers

In [25]:
g = generator(noise, keep_prob, is_training)
d_real = discriminator(X_in)
d_fake = discriminator(g, reuse=True)

print(d_real)
vars_g = [var for var in tf.trainable_variables() if 'gen' in var.name]
vars_d = [var for var in tf.trainable_variables() if 'disc' in var.name]

# Applying regularizers
d_reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_d)
g_reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_g)

print('checking d_real', d_real.shape)
print('checking d_fake', d_fake.shape)
loss_d_real = binary_cross_entropy(tf.ones_like(d_real), d_real)
loss_d_fake = binary_cross_entropy(tf.zeros_like(d_fake), d_fake)
loss_g = tf.reduce_mean(binary_cross_entropy(tf.ones_like(d_fake), d_fake))
loss_d = tf.reduce_mean(0.5 * (loss_d_real + loss_d_fake))
print(loss_d_real, loss_d_real.shape)
print(loss_d_fake, loss_d_fake.shape)

# optimizer_d = tf.train.AdamOptimizer(learning_rate).minimize(loss_d, var_list=vars_d)
# optimizer_g = tf.train.AdamOptimizer(learning_rate).minimize(loss_g, var_list=vars_g)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer_d = tf.train.RMSPropOptimizer(learning_rate=0.00015).minimize(loss_d + d_reg, var_list=vars_d)
    optimizer_g = tf.train.RMSPropOptimizer(learning_rate=0.00015).minimize(loss_g + g_reg, var_list=vars_g)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

('checking units ', 16)
Tensor("disc/dense_2/Sigmoid:0", shape=(?, 1), dtype=float32)
('checking d_real', TensorShape([Dimension(None), Dimension(1)]))
('checking d_fake', TensorShape([Dimension(None), Dimension(1)]))
(<tf.Tensor 'Neg:0' shape=(?, 1) dtype=float32>, TensorShape([Dimension(None), Dimension(1)]))
(<tf.Tensor 'Neg_1:0' shape=(?, 1) dtype=float32>, TensorShape([Dimension(None), Dimension(1)]))


# Training GAN

In [26]:
for i in range(1):
    train_d = True
    train_g = True
    keep_prob_train = 0.6
    
    # Creating noise
    n = np.random.uniform(0.0, 1.0, [batch_size, n_noise]).astype(np.float32)
    # batch = [np.reshape(b, [28, 28]) for b in mnist.train.next_batch(batch_size=batch_size)[0]]
    batch_x, _ = mnist.train.next_batch(batch_size)
    batch_x = np.reshape(batch_x, newshape=[-1, 28, 28, 1])
    print(batch_x.shape)
    #d_real_ls, d_fake_ls, g_ls, d_ls = sess.run(
    #    [loss_d_real, loss_d_fake, loss_g, loss_d], feed_dict={real_images: batch, noise: n, keep_prob: keep_prob_train, is_training: True}
    #)
    # feed_dict={real_images: batch, noise: n, keep_prob: keep_prob_train, is_training: True}
    print('chjeckin')
    # print(real_images.shape)
    d_real_ls, d_fake_ls, g_ls, d_ls = sess.run(
        [loss_d_real, loss_d_fake, loss_g, loss_d], feed_dict={X_in: batch_x, noise: n, is_training: True}
    )
    print('checking something')
    print(something)
    d_real_ls = np.mean(d_real_ls)
    d_fake_ls = np.mean(d_fake_ls)
    
    g_ls = g_ls
    d_ls = d_ls
    
    if g_ls * 1.5 < d_ls:
        train_g = False
        pass
    
    if d_ls * 2 < g_ls:
        train_d = False
        pass
    if train_d:
        sess.run(optimizer_d, feed_dict={noise: n, real_images: batch, keep_prob: keep_prob_train, is_training:True})
        
        
    if train_g:
        sess.run(optimizer_g, feed_dict={noise: n, keep_prob: keep_prob_train, is_training:True})

(64, 28, 28, 1)
chjeckin


InvalidArgumentError: Incompatible shapes: [64,1] vs. [4096,1]
	 [[Node: add_9 = Add[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Neg, Neg_1)]]

Caused by op u'add_9', defined at:
  File "/Users/jasons/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/jasons/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/jasons/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jasons/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/jasons/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/jasons/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  F